In [1]:
import polars as pl
import sqlite3
import pandas as pd
import os
import numpy as np

In [2]:
ls

0-18.db  20-24.db     Funcionou/          TentandoCriarDatabase.ipynb  tudo.db
19.db    Completo.db  merged_database.db  TrabalhoEmGrupoBD/


# Planejamento
Preciso juntar todos os dados em uma tabela que tenha ano, regiao, estado, estacao  como colunas e esteja so com informacoes diarias


In [4]:
def read_dataset_with_metadata(file_path):
    """Reads a dataset with metadata."""
    try:
        with open(file_path, 'r', encoding='latin-1') as f:
            metadata_lines = [next(f) for x in range(8)]
    except UnicodeDecodeError:
        print(f"Skipping file {file_path} due to UnicodeDecodeError.")
        return None   
    except StopIteration:
        print(f"Skipping file {file_path} due to incomplete metadata.")
        return None  

    try:
        metadata = {
            'regiao': metadata_lines[0].split(';')[1].strip(),
            'uf': metadata_lines[1].split(';')[1].strip(),
            'estacao': metadata_lines[2].split(';')[1].strip(),
            'codigo_wmo': metadata_lines[3].split(';')[1].strip(),
            'latitude': float(metadata_lines[4].split(';')[1].strip().replace(',', '.')),
            'longitude': float(metadata_lines[5].split(';')[1].strip().replace(',', '.')),
            'altitude': float(metadata_lines[6].split(';')[1].strip().replace(',', '.')),
            'data_fundacao': metadata_lines[7].split(';')[1].strip()
        }
    except (ValueError, IndexError) as e:
        print(f" {file_path}   {e}")
        return None   
     try:
        data = pd.read_csv(file_path,
                           sep=';',
                           encoding='latin-1',
                           skiprows=8,
                           decimal=',')
    except pd.errors.ParserError:
        print(f"{file_path}")
        return None   
     
    data = data.replace(-9999, np.nan)

    try:
        data[['ano', 'mes', 'dia']] = data['DATA (YYYY-MM-DD)'].str.split('-', expand=True)
    except KeyError:
        print(f"{file_path}")
 
    for key, value in metadata.items():
        data[key] = value

    aggregation_dict = {
        'HORA (UTC)': 'first',  
        'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': 'mean',
        'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)': 'mean',
        'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)': 'max',
        'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)': 'min',
        'RADIACAO GLOBAL (KJ/m²)': 'sum',
        'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': 'mean',
        'TEMPERATURA DO PONTO DE ORVALHO (°C)': 'mean',
        'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)': 'max',
        'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)': 'min',
        'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)': 'max',
        'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)': 'min',
        'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)': 'max',
        'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)': 'min',
        'UMIDADE RELATIVA DO AR, HORARIA (%)': 'mean',
        'VENTO, DIREÇÃO HORARIA (gr) (° (gr))': 'mean',
        'VENTO, RAJADA MAXIMA (m/s)': 'max',
        'VENTO, VELOCIDADE HORARIA (m/s)': 'mean',
        'Unnamed: 19': 'first',
        'ano': 'first', 
        'mes': 'first',
        'dia': 'first',
        'estacao': 'first',
        'regiao': 'first',
        'uf': 'first',
        'codigo_wmo': 'first',
        'longitude': 'first',
        'latitude': 'first',
        'altitude': 'first',
        'data_fundacao': 'first'
    }

    data = data.groupby('DATA (YYYY-MM-DD)', as_index=False).agg(aggregation_dict)

    data.columns = [
        'data',
        'hora_utc',
        'precipitacao_total_diaria',
        'pressao_media',
        'pressao_max',
        'pressao_min',
        'radiacao_total_diaria',
        'temp_ar_media',
        'temp_orvalho_media',
        'temp_max',
        'temp_min',
        'temp_orvalho_max',
        'temp_orvalho_min',
        'umidade_max',
        'umidade_min',
        'umidade_media',
        'vento_direcao_media',
        'vento_rajada_max',
        'vento_velocidade_media',
        'col_anonima',
        'ano',
        'mes',
        'dia',
        'estacao',
        'regiao',
        'uf',
        'codigo_wmo',
        'longitude',
        'latitude',
        'altitude',
        'data_fundacao'
    ]

    return {'metadata': metadata, 'data': data}


IndentationError: unindent does not match any outer indentation level (<string>, line 27)

In [ ]:
data

In [ ]:

folder_path = "TrabalhoEmGrupoBD/2016"

csv_files = [f for f in os.listdir(folder_path) if f.endswith('.CSV')]

all_data = []

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    result = read_dataset_with_metadata(file_path)

    if result is not None:
        all_data.append(result['data']) 

result = pd.concat(all_data, ignore_index=True)


In [ ]:
result

Juntando anos em uma pasta



In [ ]:
import os
import sqlite3
import pandas as pd

db_path = "0-18.db"

all_data = []
for year in range(2000, 2019):
    folder_path = f"TrabalhoEmGrupoBD/{year}"
    
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.CSV')]
    
    for file_name in csv_files:
        file_path = os.path.join(folder_path, file_name)
        result = read_dataset_with_metadata(file_path)
        
        if result is not None:
            all_data.append(result['data'])  

result = pd.concat(all_data, ignore_index=True)

conn = sqlite3.connect(db_path)

result.to_sql("result_table", conn, if_exists="replace", index=False)

conn.commit()
conn.close()


In [ ]:
result

In [ ]:
pl.read_database("SELECT * FROM result_table", conn)

## 2019 arrombado eh um ano c as colunas diferentes de tds


In [ ]:
import pandas as pd

def read_dataset_with_metadata(file_path):
    """Reads a dataset with metadata."""
    
    try:
        with open(file_path, 'r', encoding='latin-1') as f:
            metadata_lines = [next(f) for x in range(8)]
    except UnicodeDecodeError:
        print(f"{file_path}")
        return None  
    except StopIteration:
        print(f"{file_path}")
        return None  


    try:
        metadata = {
            'regiao': metadata_lines[0].split(';')[1].strip(),
            'uf': metadata_lines[1].split(';')[1].strip(),
            'estacao': metadata_lines[2].split(';')[1].strip(),
            'codigo_wmo': metadata_lines[3].split(';')[1].strip(),
            'latitude': float(metadata_lines[4].split(';')[1].strip().replace(',', '.')),
            'longitude': float(metadata_lines[5].split(';')[1].strip().replace(',', '.')),
            'altitude': float(metadata_lines[6].split(';')[1].strip().replace(',', '.')),
            'data_fundacao': metadata_lines[7].split(';')[1].strip()
        }
    except (ValueError, IndexError) as e:
        print(f"Skipping file {file_path} due to metadata parsing error: {e}")
        return None  

    try:
        data = pd.read_csv(file_path,
                           sep=';',
                           encoding='latin-1',
                           skiprows=8,
                           decimal=',')
    except pd.errors.ParserError:
        print(f"{file_path}")
        return None  
    required_columns = ['Data', 'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)', 'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)', 
                        'RADIACAO GLOBAL (KJ/m²)', 'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)', 'UMIDADE RELATIVA DO AR, HORARIA (%)']
    
    missing_columns = [col for col in required_columns if col not in data.columns]
    
    if missing_columns:
        print(f"Skipping file {file_path} due to missing columns: {', '.join(missing_columns)}")
        return None 

    try:
        data[['ano', 'mes', 'dia']] = data['Data'].str.split('/', expand=True)  
    except KeyError:
        print(f"Skipping file {file_path} due to missing 'Data' column.")
        return None  

    for key, value in metadata.items():
        data[key] = value
    aggregation_dict = {
        'Hora UTC': 'first',  
        'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': 'mean',
        'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)': 'mean',
        'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)': 'max',
        'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)': 'min',
        '''RADIACAO GLOBAL (KJ/m²)''': 'sum',
        'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': 'mean',
        'TEMPERATURA DO PONTO DE ORVALHO (°C)': 'mean',
        'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)': 'max',
        'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)': 'min',
        'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)': 'max',
        'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)': 'min',
        'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)': 'max',
        'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)': 'min',
        'UMIDADE RELATIVA DO AR, HORARIA (%)': 'mean',
        'VENTO, DIREÇÃO HORARIA (gr) (° (gr))': 'mean',
        'VENTO, RAJADA MAXIMA (m/s)': 'max',
        'VENTO, VELOCIDADE HORARIA (m/s)': 'mean',
        'Unnamed: 19': 'first',
        'ano': 'first', 
        'mes': 'first',
        'dia': 'first',
        'estacao': 'first',
        'regiao': 'first',
        'uf': 'first',
        'codigo_wmo': 'first',
        'longitude': 'first',
        'latitude': 'first',
        'altitude': 'first',
        'data_fundacao': 'first'
    }

    data = data.groupby('Data', as_index=False).agg(aggregation_dict)

   
    data.columns = [
        'data',
        'hora_utc',
        'precipitacao_total_diaria',
        'pressao_media',
        'pressao_max',
        'pressao_min',
        'radiacao_total_diaria',
        'temp_ar_media',
        'temp_orvalho_media',
        'temp_max',
        'temp_min',
        'temp_orvalho_max',
        'temp_orvalho_min',
        'umidade_max',
        'umidade_min',
        'umidade_media',
        'vento_direcao_media',
        'vento_rajada_max',
        'vento_velocidade_media',
        'col_anonima',  
        'ano',
        'mes',
        'dia',
        'estacao',
        'regiao',
        'uf',
        'codigo_wmo',
        'longitude',
        'latitude',
        'altitude',
        'data_fundacao'
    ]

    return


In [ ]:
import os
import sqlite3
import pandas as pd

db_path = "19.db"

all_data = []

for year in range(2019, 2020):
  
    folder_path = f"TrabalhoEmGrupoBD/{year}"
    
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.CSV')]
    
    for file_name in csv_files:
        file_path = os.path.join(folder_path, file_name)
        result = read_dataset_with_metadata(file_path)
        
        if result is not None:
            all_data.append(result['data'])  

result = pd.concat(all_data, ignore_index=True)

conn = sqlite3.connect(db_path)

result.to_sql("result_table", conn, if_exists="replace", index=False)

conn.commit()
conn.close()


# Agora lendo os pos 2019, que mudam algumas coisas

In [ ]:
import pandas as pd

def read_dataset_with_metadata(file_path
    try:
        with open(file_path, 'r', encoding='latin-1') as f:
            metadata_lines = [next(f) for x in range(8)]
    except UnicodeDecodeError:
        print(f"{file_path}")
        return None  
    except StopIteration:
        print(f"{file_path}")
        return None

    try:
        metadata = {
            'regiao': metadata_lines[0].split(';')[1].strip(),
            'uf': metadata_lines[1].split(';')[1].strip(),
            'estacao': metadata_lines[2].split(';')[1].strip(),
            'codigo_wmo': metadata_lines[3].split(';')[1].strip(),
            'latitude': float(metadata_lines[4].split(';')[1].strip().replace(',', '.')),
            'longitude': float(metadata_lines[5].split(';')[1].strip().replace(',', '.')),
            'altitude': float(metadata_lines[6].split(';')[1].strip().replace(',', '.')),
            'data_fundacao': metadata_lines[7].split(';')[1].strip()
        }
    except (ValueError, IndexError) as e:
        print(f"Skipping file {file_path} due to metadata parsing error: {e}")
        return None 

    try:
        data = pd.read_csv(file_path,
                           sep=';',
                           encoding='latin-1',
                           skiprows=8,
                           decimal=',')
    except pd.errors.ParserError:
        print(f" {file_path}")
        return None  

    required_columns = ['Data', 'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)', 'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)', 
                        'RADIACAO GLOBAL (Kj/m²)', 'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)', 'UMIDADE RELATIVA DO AR, HORARIA (%)']
    
    missing_columns = [col for col in required_columns if col not in data.columns]
    
    if missing_columns:
        print(f"{file_path} columns: {', '.join(missing_columns)}")
        return None  

    
    try:
        data[['ano', 'mes', 'dia']] = data['Data'].str.split('/', expand=True)
    except KeyError:
        print(f"{file_path} ")
        return None  

    for key, value in metadata.items():
        data[key] = value

    aggregation_dict = {
        'Hora UTC': 'first',  
        'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': 'mean',
        'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)': 'mean',
        'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)': 'max',
        'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)': 'min',
        '''RADIACAO GLOBAL (Kj/m²)''': 'sum',
        'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': 'mean',
        'TEMPERATURA DO PONTO DE ORVALHO (°C)': 'mean',
        'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)': 'max',
        'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)': 'min',
        'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)': 'max',
        'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)': 'min',
        'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)': 'max',
        'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)': 'min',
        'UMIDADE RELATIVA DO AR, HORARIA (%)': 'mean',
        'VENTO, DIREÇÃO HORARIA (gr) (° (gr))': 'mean',
        'VENTO, RAJADA MAXIMA (m/s)': 'max',
        'VENTO, VELOCIDADE HORARIA (m/s)': 'mean',
        'Unnamed: 19': 'first',
        'ano': 'first', 
        'mes': 'first',
        'dia': 'first',
        'estacao': 'first',
        'regiao': 'first',
        'uf': 'first',
        'codigo_wmo': 'first',
        'longitude': 'first',
        'latitude': 'first',
        'altitude': 'first',
        'data_fundacao': 'first'
    }

    data = data.groupby('Data', as_index=False).agg(aggregation_dict)

    data.columns = [
        'data',
        'hora_utc',
        'precipitacao_total_diaria',
        'pressao_media',
        'pressao_max',
        'pressao_min',
        'radiacao_total_diaria',
        'temp_ar_media',
        'temp_orvalho_media',
        'temp_max',
        'temp_min',
        'temp_orvalho_max',
        'temp_orvalho_min',
        'umidade_max',
        'umidade_min',
        'umidade_media',
        'vento_direcao_media',
        'vento_rajada_max',
        'vento_velocidade_media',
        'col_anonima', 
        'ano',
        'mes',
        'dia',
        'estacao',
        'regiao',
        'uf',
        'codigo_wmo',
        'longitude',
        'latitude',
        'altitude',
        'data_fundacao'
    ]

    return {'metadata': metadata, 'data': data}


In [ ]:
import os
import sqlite3
import pandas as pd

db_path = "20-24.db"

all_data = []

for year in range(2020, 2025):
    folder_path = f"TrabalhoEmGrupoBD/{year}"
    
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.CSV')]
    
    for file_name in csv_files:
        file_path = os.path.join(folder_path, file_name)
        result = read_dataset_with_metadata(file_path)
        
        if result is not None:
            all_data.append(result['data'])  

result = pd.concat(all_data, ignore_index=True)

conn = sqlite3.connect(db_path)

result.to_sql("result_table", conn, if_exists="replace", index=False)

conn.commit()
conn.close()


In [ ]:
ls

In [ ]:
import sqlite3
import pandas as pd

def load_data_from_db(db_file):
    conn = sqlite3.connect(db_file)
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    tables = pd.read_sql(query, conn)
    print(f"Tables in {db_file}: {tables}")
    
    table_name = tables.iloc[0, 0]
    df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
    conn.close()
    return df

def save_data_to_new_db(new_db_file, df, table_name):
    conn = sqlite3.connect(new_db_file)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()

new_db_file = 'merged_database.db'

df_0_18 = load_data_from_db('0-18.db')
df_19 = load_data_from_db('19.db')
df_20_24 = load_data_from_db('20-24.db')

save_data_to_new_db(new_db_file, df_0_18, 'table_0_18')
save_data_to_new_db(new_db_file, df_19, 'table_19')
save_data_to_new_db(new_db_file, df_20_24, 'table_20_24')

print(f" {new_db_file}")


In [ ]:
conn = sqlite3.connect('0-18.db')
conn1 = sqlite3.connect('19.db')
conn2 = sqlite3.connect('20-24.db')

cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

tables = cursor.fetchall()
for table in tables:
    print(table[0])




In [ ]:
pl.read_database("SELECT * FROM result_table",conn)

In [ ]:
import sqlite3

conn_0_18 = sqlite3.connect('0-18.db')
conn_19 = sqlite3.connect('19.db')
conn_20_24 = sqlite3.connect('20-24.db')

conn_merged = sqlite3.connect('merged_database.db')
cursor_merged = conn_merged.cursor()

cursor_0_18 = conn_0_18.cursor()
cursor_0_18.execute("SELECT name FROM sqlite_master WHERE type='table';")
table_name = cursor_0_18.fetchone()[0]  

cursor_0_18.execute(f"PRAGMA table_info({table_name});")
columns = [col[1] for col in cursor_0_18.fetchall()]  
columns_str = ", ".join(columns) 

cursor_merged.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ({columns_str});")

def copy_data(source_conn, source_table, target_conn):
    source_cursor = source_conn.cursor()
    source_cursor.execute(f"SELECT * FROM {source_table}")
    rows = source_cursor.fetchall()

    placeholders = ", ".join(["?"] * len(columns))
    target_cursor = target_conn.cursor()
    target_cursor.executemany(f"INSERT INTO {source_table} VALUES ({placeholders})", rows)
    target_conn.commit()

copy_data(conn_0_18, table_name, conn_merged)
copy_data(conn_19, table_name, conn_merged)
copy_data(conn_20_24, table_name, conn_merged)

conn_0_18.close()
conn_19.close()
conn_20_24.close()
conn_merged.close()



In [ ]:
conn = sqlite3.connect('merged_database.db')
pl.read_database("SELECT * FROM result_table ",conn)

In [ ]:
conn.close()